## Importing libraries

In [ ]:
import requests
import pandas as pd
import json
import numpy as np
import datetime as dt

In [2]:
weather_df = pd.read_csv('../data/weather/weather_central_park.csv')

/var/folders/tx/byxnmbjd375b6lk48n6v_1bc0000gn/T/ipykernel_18367/2014505126.py:1: DtypeWarning: Columns (36,42,43,48,52,88) have mixed types. Specify dtype option on import or set low_memory=False.
  weather_df = pd.read_csv('../data/weather_central_park.csv')


## Processing dataset

Filtering data for our target period

In [4]:
weather_df = weather_df[(weather_df['DATE'] >= '2023-01-01') & (weather_df['DATE'] <= '2023-12-31')]

Selecting and scaling data

In [5]:
# Get the hourly weather report type
df = weather_df.loc[weather_df['REPORT_TYPE'] == 'FM-15', :]

# Extract the unscaled values for each column
df.loc[:,'WND'] = (df['WND'].apply(lambda x: int(x.split(',')[-2])/10)
                            .replace(999.9, np.nan))

df.loc[:,'TMP'] = (df['TMP'].apply(lambda x: int(x.split(',')[0])/10)
                            .replace(999.9, np.nan))

df.loc[:,'DEW'] = (df['DEW'].apply(lambda x: int(x.split(',')[0])/10)
                            .replace(999.9, np.nan))

df.loc[:,'SLP'] = (df['SLP'].apply(lambda x: int(x.split(',')[0])/10)
                            .replace(9999.9, np.nan))

# Impute missing data using data from an hour before
df.ffill(inplace=True)

# Filter data to period between 2023-01 to 2023-12
processed_data = df.loc[(df['DATE'] >= '2023-01-01') & (df['DATE'] <= '2023-12-31'), :]

# Extract date and hour from datetime column
processed_data.loc[:,'date'] = pd.to_datetime(processed_data['DATE'])
processed_data.loc[:,'hour'] = processed_data['date'].dt.hour
processed_data.loc[:,'date'] = processed_data['date'].dt.date

processed_data.rename({'WND':'wnd',
                        'TMP':'tmp',
                        'DEW':'dew',
                        'SLP':'atm'},
                        axis=1,
                        inplace=True)


processed_df =  processed_data[['date', 'hour', 'tmp', 'dew', 'atm']]

# These preprocessing steps are inspired by Ming Hui Tan, 2022 MAST30034 Assignment 1, preprocessing jupyter notebook

/var/folders/tx/byxnmbjd375b6lk48n6v_1bc0000gn/T/ipykernel_18367/2733000853.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.ffill(inplace=True)
/var/folders/tx/byxnmbjd375b6lk48n6v_1bc0000gn/T/ipykernel_18367/2733000853.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.ffill(inplace=True)


In [10]:
processed_df.head()

,date,hour,tmp,dew,atm
5,2023-01-01,0,10.0,9.4,1009.8
8,2023-01-01,1,12.8,11.7,1008.6
10,2023-01-01,2,12.2,11.7,1008.6
13,2023-01-01,3,12.8,11.1,1007.8
16,2023-01-01,4,12.2,11.1,1007.0


Writing processed dataset to `data/weather` directory

In [9]:
processed_df.to_csv('../data/weather/hourly_weather.csv')